In [1]:
import tensorflow as tf
import numpy as np

In [9]:
class LinearModel(object):
    def __init__(
        self,
        num_features,
    ):
        self.num_features = num_features
        with tf.variable_scope(
            'LinearModelParams',
            reuse=False,
            initializer=tf.random_uniform_initializer(
                -1.0, 1.0
            ),
        ):
            self.b0 = tf.get_variable(
                'b0',
                [],
            )
            self.b1 = tf.get_variable(
                'b1',
                [num_features],
            )
        self.saver = tf.train.Saver({
            'b0': self.b0,
            'b1': self.b1,
        })
        tf.summary.scalar('b0', self.b0)
        tf.summary.histogram('b1', self.b1)
    
    def make_training_graph(
        self,
        batch_size,
    ):
        with tf.name_scope('placeholders'):
            X_train = tf.placeholder(
                shape=[batch_size, self.num_features],
                name='X_train',
                dtype=tf.float32,
            )
            y_train = tf.placeholder(
                shape=[batch_size],
                name='y_train',
                dtype=tf.float32,
            )
        with tf.name_scope('model'):
            predictions = tf.identity(
                tf.squeeze(tf.matmul(X_train, tf.expand_dims(self.b1, 1))) + self.b0,
                name='predictions',
            )
            residuals = tf.identity(
                y_train - predictions,
                name='residuals'
            )
            tf.summary.histogram('residuals', residuals)
            loss = tf.reduce_mean(
                tf.square(residuals),
                name='loss',
            )
            tf.summary.scalar('loss', loss)
        with tf.name_scope('train_ops'):
            optimizer = tf.train.GradientDescentOptimizer(0.01)
            train_op = optimizer.minimize(loss)
        return {
            'placeholders': {
                'X_train': X_train,
                'y_train': y_train,
            },
            'outputs': {
                'predictions': predictions,
                'residuals': residuals,
                'loss': loss,
            },
            'train_ops': {
                'train_op': train_op,
                'summary': tf.summary.merge_all(),
            }
        }
    
    def save(
        self,
        sess,
        path,
    ):
        self.saver.save(sess, path)

    def restore(
        self,
        sess,
        path,
    ):
        self.saver.restore(sess, path)

In [10]:
tf.reset_default_graph()
lm = LinearModel(5)

In [11]:
graph = lm.make_training_graph(16)

In [12]:
Xtr = np.random.uniform(-10., 10., size=(16,5))
true_values = np.random.uniform(-1., 1., size=(5,))
true_bias = np.random.uniform(-1., 1.)
ytr = np.random.uniform(-0.25, 0.25, size=(Xtr.shape[0],)) + np.dot(Xtr, true_values) + true_bias

true_bias, true_values

(0.5721726802941178,
 array([-0.23099213,  0.65407194, -0.56684004,  0.46271707,  0.95261418]))

In [13]:
logdir = 'logs/{0:08X}'.format(np.random.randint(2**32))
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logdir, sess.graph)
    print(lm.b0.eval())
    print(lm.b1.eval())
    for i in range(100000):
        outputs = sess.run(graph['train_ops'], feed_dict={
            graph['placeholders']['X_train']: Xtr,
            graph['placeholders']['y_train']: ytr,
        })
        writer.add_summary(outputs['summary'], i)
    lm.save(sess, '{0}/model.ckpt'.format(logdir))

-0.422751
[-0.46575141  0.39161229 -0.40420103  0.87393165 -0.29090929]


In [7]:
with tf.Session() as sess:
    lm.restore(sess, '{0}/model.ckpt'.format(logdir))
    outputs = sess.run(graph['outputs'], feed_dict={
        graph['placeholders']['X_train']: Xtr,
        graph['placeholders']['y_train']: ytr,
    })
    print(outputs)
    print(lm.b0.eval())
    print(lm.b1.eval())

INFO:tensorflow:Restoring parameters from logs/3DD28439/model.ckpt
{'predictions': array([  2.53248596,  -8.86022854,   8.41758156,  -9.00340939,
        -4.54157305,   3.40779209,  -2.74037886,  -8.44541168,
        11.8639307 ,   1.12247264,  -3.21660566,   7.60387087,
        13.67782211,  -9.36620617,   7.19500256,   6.12647295], dtype=float32), 'residuals': array([ 0.21096492, -0.03359222, -0.1590147 , -0.01538277, -0.09021139,
       -0.13335085, -0.11556268,  0.05635548, -0.09206867, -0.14584553,
        0.2226119 ,  0.21950436, -0.06024647, -0.05427647,  0.02450132,
        0.16559029], dtype=float32), 'loss': 0.017230622, 'summary': b'\n\t\n\x02b0\x15\xba\xb0\x0c\xbf\n\xd8\x01\n\x02b1*\xd1\x01\t\x00\x00\x00\x00\xc0j\xe9\xbf\x11\x00\x00\x00\x00\xa2\x1b\xed?\x19\x00\x00\x00\x00\x00\x00\x14@!\x00\x00\x00\x00\xf1\xcc\xd6\xbf)H?e\xd5\xfa\n\xfc?2P\x1e\x8b\x8e\xe8\xfb\xae\xea\xbf2g\xc7G\xfcA\xe8\xbf\xda^\xbd\xaeh\xe5\xd8\xbf\x97\xca7\x13\x02\xa2\xd6\xbf\xcfCa\xb4G\x93\xd4\xbf\x05\xc7